In [72]:
import os
import sys
import glob
import keras
from keras.models import Sequential
from tensorflow.keras.utils import img_to_array, array_to_img, load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
K.set_image_data_format('channels_first')

In [73]:
# default settings
img_width, img_height = 224, 224

train_dir = '/Users/Janeshelbyporter/Documents/OneDrive-CofC/BE/Image-Recognition-with-Deep-Learning-master/data/noodles/train'
validate_dir = '/Users/Janeshelbyporter/Documents/OneDrive-CofC/BE/Image-Recognition-with-Deep-Learning-master/data/noodles/validate'
epochs = 5
batch_size = 32
num_classes = 2

In [74]:
# get number of images in training directory
nb_train_samples = 0
for r, dirs, files in os.walk(train_dir):
    for dr in dirs:
        nb_train_samples += len(glob.glob(os.path.join(r, dr + "/*")))
print(nb_train_samples)

315


In [75]:
# get number of images in validation directory
nb_validate_samples = 0
for r, dirs, files in os.walk(validate_dir):
    for dr in dirs:
        nb_validate_samples += len(glob.glob(os.path.join(r, dr + "/*")))
print(nb_validate_samples)

36


In [76]:
# data pre-processing for training
train_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

In [77]:
# data pre-processing for validation
validate_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

In [78]:
# generate and store training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

Found 315 images belonging to 2 classes.


In [79]:
# generate and store validation data
validate_generator = validate_datagen.flow_from_directory(
    validate_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

Found 36 images belonging to 2 classes.


In [80]:
# define the model
def convnet_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
                    input_shape=(3, img_width, img_height),
                    activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same',
                    activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same',
                    activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [81]:
# define learning rate schedule
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

In [82]:
# instantiate CNN model
model_final = convnet_model()

In [83]:
# Train the model
lr = 0.01
sgd = SGD(learning_rate=lr, decay=1e-6, momentum=0.9, nesterov=True)
model_final.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [84]:
model_final.fit(
    train_generator,
    epochs = epochs,
# #     steps_per_epoch = nb_train_samples / batch_size,
    validation_data = validate_generator,
#     validation_steps = nb_validate_samples / batch_size,
    class_weight={0: 10., 1: 1.},
    callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/5


2022-10-18 09:25:43.697027: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - ETA: 0s - loss: 2.4252 - accuracy: 0.4698

2022-10-18 09:26:00.737943: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 18s 1s/step - loss: 2.4252 - accuracy: 0.4698 - val_loss: 1.0405 - val_accuracy: 0.4444 - lr: 0.0100
Epoch 2/5
10/10 [==============================] - 11s 1s/step - loss: 1.8179 - accuracy: 0.4476 - val_loss: 1.2974 - val_accuracy: 0.4444 - lr: 0.0100
Epoch 3/5
10/10 [==============================] - 11s 1s/step - loss: 1.8026 - accuracy: 0.4476 - val_loss: 1.3981 - val_accuracy: 0.4444 - lr: 0.0100
Epoch 4/5
10/10 [==============================] - 12s 1s/step - loss: 1.7739 - accuracy: 0.4476 - val_loss: 1.1289 - val_accuracy: 0.4444 - lr: 0.0100
Epoch 5/5
10/10 [==============================] - 11s 1s/step - loss: 1.7700 - accuracy: 0.4476 - val_loss: 1.4617 - val_accuracy: 0.4444 - lr: 0.0100
